# Requirements

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, KFold, RepeatedKFold, GridSearchCV, cross_val_predict, TimeSeriesSplit
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.metrics import classification_report, accuracy_score, f1_score, confusion_matrix, mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBClassifier, XGBRegressor

# Laboratory Exercise - Run Mode (8 points)

## Introduction
In this laboratory assignment, the focus is on time series forecasting, specifically targeting the prediction of the current **mean temperature** in the city of Delhi. Your task involves employing bagging and boosting methods to forecast the **mean temperature**. To accomplish this use data from the preceding three days, consisting of **mean temperature**, **humidity**, **wind speed**, and **mean pressure**.

**Note: You are required to perform this laboratory assignment on your local machine.**

## The Climate Dataset

## Downloading the Climate Dataset

## Exploring the Climate Dataset
This dataset consists of daily weather records for the city of Delhi spanning a period of 4 years (from 2013 to 2017). The dataset includes the following attributes:

- date - date in the format YYYY-MM-DD,
- meantemp - mean temperature averaged from multiple 3-hour intervals in a day,
- humidity - humidity value for the day (measured in grams of water vapor per cubic meter volume of air),
- wind_speed - wind speed measured in kilometers per hour, and
- meanpressure - pressure reading of the weather (measured in atm).

*Note: The dataset is complete, with no missing values in any of its entries.*

Load the dataset into a `pandas` data frame.

In [3]:
data = pd.read_csv('climate-data.csv')
data.sample(10)

,date,meantemp,humidity,wind_speed,meanpressure
546,2014-07-01,31.375000,65.125000,6.250000,1000.875000
1224,2016-05-09,33.533333,40.600000,18.425000,1006.062500
334,2013-12-01,18.000000,65.000000,1.585714,1016.142857
1251,2016-06-05,36.166667,51.750000,8.500000,1002.833333
775,2015-02-15,18.875000,70.125000,1.387500,1012.625000
1145,2016-02-20,23.625000,73.125000,11.118750,1014.125000
155,2013-06-05,37.166667,36.500000,9.866667,998.000000
1459,2016-12-30,14.095238,89.666667,6.266667,1017.904762
1020,2015-10-18,26.857143,71.714286,11.112500,1013.125000
639,2014-10-02,30.375000,53.375000,2.787500,1009.125000


Explore the dataset using visualizations of your choice.

In [5]:
data[['meantemp', 'humidity', 'wind_speed', 'meanpressure']].corr()

,meantemp,humidity,wind_speed,meanpressure
meantemp,1.000000,-0.571951,0.306468,-0.038818
humidity,-0.571951,1.000000,-0.373972,0.001734
wind_speed,0.306468,-0.373972,1.000000,-0.020670
meanpressure,-0.038818,0.001734,-0.020670,1.000000


# Feauture Extraction
Apply a lag of one, two, and three days to each feature, creating a set of features representing the meteorological conditions from the previous three days. To maintain dataset integrity, eliminate any resulting missing values at the beginning of the dataset.

Hint: Use `df['column_name'].shift(period)`. Check the documentation at https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.shift.html.

In [7]:
data = data.set_index('date')
data = data.sort_index()
data

,meantemp,humidity,wind_speed,meanpressure
date,,,,
2013-01-01,10.000000,84.500000,0.000000,1015.666667
2013-01-02,7.400000,92.000000,2.980000,1017.800000
2013-01-03,7.166667,87.000000,4.633333,1018.666667
2013-01-04,8.666667,71.333333,1.233333,1017.166667
2013-01-05,6.000000,86.833333,3.700000,1016.500000
...,...,...,...,...
2016-12-28,17.217391,68.043478,3.547826,1015.565217
2016-12-29,15.238095,87.857143,6.000000,1016.904762
2016-12-30,14.095238,89.666667,6.266667,1017.904762


In [9]:
lag = 3
for i in range(1, lag+1):
    data[f'meantemp_prev_{i}'] = data['meantemp'].shift(i)

In [11]:
data

,meantemp,humidity,wind_speed,meanpressure,meantemp_prev_1,meantemp_prev_2,meantemp_prev_3
date,,,,,,,
2013-01-01,10.000000,84.500000,0.000000,1015.666667,NaN,NaN,NaN
2013-01-02,7.400000,92.000000,2.980000,1017.800000,10.000000,NaN,NaN
2013-01-03,7.166667,87.000000,4.633333,1018.666667,7.400000,10.000000,NaN
2013-01-04,8.666667,71.333333,1.233333,1017.166667,7.166667,7.400000,10.000000
2013-01-05,6.000000,86.833333,3.700000,1016.500000,8.666667,7.166667,7.400000
...,...,...,...,...,...,...,...
2016-12-28,17.217391,68.043478,3.547826,1015.565217,16.850000,17.142857,14.000000
2016-12-29,15.238095,87.857143,6.000000,1016.904762,17.217391,16.850000,17.142857
2016-12-30,14.095238,89.666667,6.266667,1017.904762,15.238095,17.217391,16.850000


In [13]:
data = data.dropna(axis=0)
data

,meantemp,humidity,wind_speed,meanpressure,meantemp_prev_1,meantemp_prev_2,meantemp_prev_3
date,,,,,,,
2013-01-04,8.666667,71.333333,1.233333,1017.166667,7.166667,7.400000,10.000000
2013-01-05,6.000000,86.833333,3.700000,1016.500000,8.666667,7.166667,7.400000
2013-01-06,7.000000,82.800000,1.480000,1018.000000,6.000000,8.666667,7.166667
2013-01-07,7.000000,78.600000,6.300000,1020.000000,7.000000,6.000000,8.666667
2013-01-08,8.857143,63.714286,7.142857,1018.714286,7.000000,7.000000,6.000000
...,...,...,...,...,...,...,...
2016-12-28,17.217391,68.043478,3.547826,1015.565217,16.850000,17.142857,14.000000
2016-12-29,15.238095,87.857143,6.000000,1016.904762,17.217391,16.850000,17.142857
2016-12-30,14.095238,89.666667,6.266667,1017.904762,15.238095,17.217391,16.850000


## Dataset Splitting
Partition the dataset into training and testing sets with an 80:20 ratio.

**WARNING: DO NOT SHUFFLE THE DATASET.**



In [15]:
features = [f'meantemp_prev_{i}' for i in range(1, lag+1)]

## Ensemble Learning Methods

### Bagging

Create an instance of a Random Forest model and train it using the `fit` function.

In [17]:
X, Y = data[features], data.meantemp
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, shuffle=False)

rf_model = RandomForestRegressor(n_estimators=30, max_depth=3, criterion='squared_error')
rf_model.fit(X_train, Y_train)

RandomForestRegressor(max_depth=3, n_estimators=30)

Use the trained model to make predictions for the test set.

In [19]:
y_pred = rf_model.predict(X_test)
y_pred

array([23.74367981, 23.74367981, 26.59701619, 26.59701619, 27.28128136,
       27.28128136, 27.28128136, 24.86974644, 26.59701619, 28.50779583,
       30.49227164, 24.96275665, 26.59701619, 26.59701619, 27.28128136,
       30.02425861, 30.49227164, 30.49227164, 31.98923102, 33.07846188,
       32.71461511, 31.98923102, 31.12236114, 30.49227164, 29.91891259,
       30.57212732, 31.98923102, 30.36604651, 30.49227164, 31.12236114,
       32.94639886, 33.39396315, 34.29062006, 33.63903914, 33.63903914,
       33.50942899, 33.50942899, 33.50942899, 33.07846188, 31.34151341,
       31.12236114, 31.12236114, 33.39396315, 32.94639886, 33.50942899,
       33.50942899, 32.94639886, 33.63903914, 34.29062006, 34.29062006,
       33.50942899, 30.49227164, 31.12236114, 33.07846188, 34.29062006,
       33.39396315, 33.50942899, 32.22241067, 34.29062006, 34.29062006,
       34.29062006, 34.29062006, 34.29062006, 34.29062006, 34.29062006,
       34.29062006, 34.29062006, 34.29062006, 34.29062006, 31.98

Assess the performance of the model by using different metrics provided by the `scikit-learn` library.

In [21]:
r2_score(Y_test, y_pred)

0.9042354292970198

### Boosting

Create an instance of an XGBoost model and train it using the `fit` function.

In [23]:
xgb_model = XGBRegressor(n_estimators=35, max_depth=15, learning_rate=0.1, objective='reg:squarederror')
xgb_model.fit(X_train, Y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=15, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=35, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

Use the trained model to make predictions for the test set.

In [25]:
y_pred_xgb = xgb_model.predict(X_test)

Assess the performance of the model by using different metrics provided by the `scikit-learn` library.

In [27]:
mse = mean_squared_error(Y_test, y_pred_xgb)
mae = mean_absolute_error(Y_test, y_pred_xgb)
r2 = r2_score(Y_test, y_pred_xgb)

print('Mean square error:', mae)
print('Mean absolute error: ', mae)
print('R2 score: ', r2)

Mean square error: 1.484412610406554
Mean absolute error:  1.484412610406554
R2 score:  0.8860090592400672


# Laboratory Exercise - Bonus Task (+ 2 points)

As part of the bonus task in this laboratory assignment, your objective is to fine-tune the number of estimators (`n_estimators`) for the XGBoost model using a cross-validation with grid search and time series split. This involves systematically experimenting with various values for `n_estimators` and evaluating the model's performance using cross-validation. Upon determining the most suitable `n_estimators` value, evaluate the model's performance on a test set for final assessment.

Hints:
- For grid search use the `GridCVSearch` from the `scikit-learn` library. Check the documentation at https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html.
- For cross-validation use the `TimeSeriesSplit` from the `scikit-learn` library. Check the documentation at https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.TimeSeriesSplit.html.

## Dataset Splitting
Partition the dataset into training and testing sets with an 90:10 ratio.

**WARNING: DO NOT SHUFFLE THE DATASET.**

In [29]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, shuffle=False)

## Fine-tuning the XGBoost Hyperparameter
Experiment with various values for `n_estimators` and evaluate the model's performance using cross-validation.

In [33]:
tscv = TimeSeriesSplit(n_splits=7)
model_xgb = XGBRegressor()
param_grid = {'n_estimators': [35, 40, 45, 50, 60, 70, 100]}
grid_search = GridSearchCV(
    estimator=model_xgb,
    param_grid= param_grid,
    cv = tscv, 
    scoring='neg_mean_squared_error'
)
grid_search.fit(X_train, Y_train)
best_n_estimator = grid_search.best_params_['n_estimators']
print(f"Best n_estimator: {best_n_estimator}")

Best n_estimator: 35


## Final Assessment of the Model Performance
Upon determining the most suitable `n_estimators` value, evaluate the model's performance on a test set for final assessment.

In [35]:
final_model = XGBRegressor(n_estimators=best_n_estimator, random_state=42)
final_model.fit(X_train, Y_train)

y_preds = final_model.predict(X_test)

mse = mean_squared_error(Y_test, y_preds)
mae = mean_absolute_error(Y_test, y_preds)
r2 = r2_score(Y_test, y_preds)

print(f"Final Model Performance:")
print(f"  Mean Squared Error (MSE): {mse:.4f}")
print(f"  Mean Absolute Error (MAE): {mae:.4f}")
print(f"  R-squared (R2 Score): {r2:.4f}")

Final Model Performance:
  Mean Squared Error (MSE): 2.0238
  Mean Absolute Error (MAE): 1.0845
  R-squared (R2 Score): 0.9379
